In [39]:
from nbdev import *

In [40]:
%nbdev_hide
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Create JNK artifact from JNK data "before handling missing"

In [41]:
import pandas as pd
import numpy as np
from fastcore.all import *
import wandb
from datetime import datetime, timedelta
from timecluster_extension.load import *

In [42]:
base_path_JNK = Path.home()/'data/PACMEL-2019/JNK/' # *
[path for path in base_path_JNK.ls()]

[Path('/home/user/data/PACMEL-2019/JNK/df_bool_dict.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/df_categorical_dict.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/df_jnk_base.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_before_handling_missing.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing_clusters_nk.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing_clusters_npk.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing_tslearn_cutt_0.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing_tslearn_cutt_mean.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolate_nothing_tslearn_mul_0.pickle'),
 Path('/home/user/data/PACMEL-2019/JNK/jnk_filled_removelong_imputeinterpolat

In [43]:
jnk_files = []
filepaths_pickle = base_path_JNK.ls(file_exts='.pickle')
for i, path in enumerate(filepaths_pickle):
    f = open(path, 'rb')    
    bin_data = f.read()
    print(f'Loading file {i}...')
    jnk_files.append(pickle.loads(bin_data))

Loading file 0...
Loading file 1...
Loading file 2...
Loading file 3...
Loading file 4...
Loading file 5...
Loading file 6...
Loading file 7...
Loading file 8...
Loading file 9...
Loading file 10...
Loading file 11...
Loading file 12...
Loading file 13...


In [44]:
df = jnk_files[3][0]
df.index.freq = 's'

In [45]:
df = df.interpolate(limit_direction='both')

In [46]:
# Note: normalization is now done when creating the artifact, to store the means and stds in the metadata
#df = normalize_columns(df)

In [47]:
df = df.resample('5s').mean()

In [48]:
df = df[432000:518400]

In [49]:
#17280 muestras = 1 dia
# 10 dias = 
# training df[0:311040]     (18dias)     Normalize
# validation df[311040:432000]  (7dias)  Don't normalize
# testing df[432000:518400]    (5dias)  Don't normalize

In [50]:
df

,RCD_AverageThree-phaseCurrent,LCD_AverageThree-phaseCurrent,LP_AverageThree-phaseCurrent,LHD_LeftHaulageDrive(tractor)Temperature(gearbox),RHD_RightHaulageDrive(tractor)Temperature(gearbox),LA_LeftArmTemperature,RA_RightArmTemperature,SM_DailyRouteOfTheShearer,SM_TotalRoute,LHD_EngineCurrent,RHD_EngineCurrent,RCD_BearingTemperature,SM_ShearerSpeed,SM_ShearerLocation,SM_ShearerMoveInLeft,SM_ShearerMoveInRight
TIMESTAMP,,,,,,,,,,,,,,,,
2019-06-26 00:00:00,14.0,11.8,0.0,56.0,0.0,51.0,50.4,1374.8,181.9,0.0,0.0,58.0,0.0,114.0,0.0,0.0
2019-06-26 00:00:05,14.0,11.8,0.0,56.0,0.0,51.0,50.0,0.0,181.9,0.0,0.0,57.0,0.0,114.0,0.0,0.0
2019-06-26 00:00:10,14.0,12.0,0.0,56.0,0.0,51.0,50.4,0.0,181.9,0.0,0.0,57.0,0.0,114.0,0.0,0.0
2019-06-26 00:00:15,14.0,12.0,0.0,56.0,0.0,51.0,50.0,0.0,181.9,0.0,0.0,57.0,0.0,114.0,0.0,0.0
2019-06-26 00:00:20,14.0,12.0,0.0,56.0,0.0,51.0,50.2,0.0,181.9,0.0,0.0,57.0,0.0,114.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-29 23:59:40,13.0,11.0,0.0,73.0,0.0,46.0,48.0,758.0,192.1,0.0,0.0,58.0,0.0,177.0,0.0,0.0
2019-06-29 23:59:45,13.0,11.0,0.0,73.0,0.0,46.0,48.0,758.0,192.1,0.0,0.0,58.0,0.0,177.0,0.0,0.0
2019-06-29 23:59:50,13.0,11.0,0.0,73.0,0.0,46.0,48.0,758.0,192.1,0.0,0.0,58.0,0.0,177.0,0.0,0.0


In [51]:
ruta_wandb_artifacts = 'data/PACMEL-2019/wandb_artifacts'# When permissions are fixed, save artifacts in 'data/PACMEL-2019/wandb_artifacts'
ar = TSArtifact.from_df(df, 'JNK', normalize=False, path= str(Path.home()/ruta_wandb_artifacts))

In [52]:
ar.metadata

{'TS': {'sd': '2019-06-26 00:00:00',
  'ed': '2019-06-30 00:00:00',
  'created': 'from-df',
  'freq': '<5 * Seconds>',
  'n_vars': 16,
  'n_samples': 69121,
  'has_missing_values': 'False',
  'vars': ['RCD_AverageThree-phaseCurrent',
   'LCD_AverageThree-phaseCurrent',
   'LP_AverageThree-phaseCurrent',
   'LHD_LeftHaulageDrive(tractor)Temperature(gearbox)',
   'RHD_RightHaulageDrive(tractor)Temperature(gearbox)',
   'LA_LeftArmTemperature',
   'RA_RightArmTemperature',
   'SM_DailyRouteOfTheShearer',
   'SM_TotalRoute',
   'LHD_EngineCurrent',
   'RHD_EngineCurrent',
   'RCD_BearingTemperature',
   'SM_ShearerSpeed',
   'SM_ShearerLocation',
   'SM_ShearerMoveInLeft',
   'SM_ShearerMoveInRight'],
  'hash': '-9095578115914338954'}}

In [53]:
run = wandb.init(entity = 'pacmel', project='timecluster-extension', job_type='create_dataset', resume=True)
run.log_artifact(ar,aliases=['test_5days'])
run.finish()

In [54]:
ar.__class__

timecluster_extension.load.TSArtifact

In [55]:
wandb.__version__

'0.10.9'